# 7 函数是一等对象

Python并不是函数式语言

一等对象定义
- 在运行时创建
- 能赋值给变量或数据结构中的元素
- 能作为参数传给函数
- 能作为函数的返回结果

## 7.2 把函数视为对象

In [3]:
def factorial(n):
    """returns n!"""
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)
    
print(factorial(5))
print(factorial.__doc__)
print(type(factorial))

fact = factorial
print(fact)
print(fact(5))
# map(function, iterable)调用会返回一个可迭代对象，所含的项是把第二个参数（可迭代对象）中的每个元素都传入第一个参数（函数）中得到的结果
print(map(factorial, range(11)))
print(list(map(fact, range(11))) )

120
returns n!
<class 'function'>
<function factorial at 0x1079b2d40>
120
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


## 7.3 高阶函数

接受函数为参数，或者把函数作为结果返回的函数是高阶函数（higher-order function）
map就是一个高阶函数， sorted也是一个高阶函数

In [6]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
print(sorted(fruits, key=len))

def reverse(word):
    return word[::-1]

print(reverse('testing'))

print(sorted(fruits, key=reverse))

['fig', 'apple', 'cherry', 'banana', 'raspberry', 'strawberry']
gnitset
['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']


In [9]:
# map，filter和reduce的替代品
def factorial(n):
    """returns n!"""
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

print(list(map(factorial, range(6))))
print([factorial(n) for n in range(6)])

print(list(map(factorial, filter(lambda n: n % 2, range(6)))))
print([factorial(n) for n in range(6) if n % 2])

from functools import reduce
from operator import add
print(reduce(add, range(100)))
print(sum(range(100)))

[1, 1, 2, 6, 24, 120]
[1, 1, 2, 6, 24, 120]
[1, 6, 120]
[1, 6, 120]
4950
4950


## 7.4 匿名函数

lambda关键字在Python表达式内创建匿名函数,lambda函数的主体只能是纯粹的表达式。

In [10]:
fruits = ['strawberry', 'fig', 'apple', 'cherry', 'raspberry', 'banana']
print(sorted(fruits, key=lambda word: word[::-1]))


['banana', 'apple', 'fig', 'raspberry', 'strawberry', 'cherry']


## 7.5 9种可调用对象

想判断对象能否调用，可以使用内置的callable()函数

- 用户定义的函数
    使用def或lambda创建
- 内置函数
    使用C语言（CPython）实现的函数，如len或time.strftime
- 内置方法
    使用C语言实现的方法，如dict.get
- 方法
    在类的定义体中定义的函数
- 类
    调用类时运行类的__new__方法创建一个实例，然后运行__init__方法，初始化实例，最后把实例返回给调用方
- 类的实例
    如果类定义了__call__方法，那么它的实例可以作为函数调用
- 生成器函数
    使用yield关键字的函数或方法。调用生成器函数返回的是生成器对象
- 协程
    使用async def定义的函数，调用协程函数会返回一个协程对象
- 异步生成器
    使用async def定义，并且使用yield关键字的函数

生成器、协程和异步生成器的返回值不是数据，而是需要进一步处理的对象。
生成器函数会返回迭代器。
原生协程和异步生成器函数返回的对象只能由异步编程框架（如asyncio）使用。

In [12]:
print([callable(obj) for obj in (abs, str, 13)])

[True, True, False]


## 7.6 用户定义的可调用类型

任何Python对象都可以表现得像函数。只需实现实例方法__call__。

In [13]:
import random
class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        # 打乱顺序
        random.shuffle(self._items)
        
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
        
    def __call__(self):
        return self.pick()
    
bingo = BingoCage(range(3))
print(bingo.pick())
print(bingo())

2
0


## 7.7 从位置参数到仅限关键字参数

In [14]:
# 使用class_作为关键字参数，是为了避免与Python关键字class发生冲突
def tag(name, *content, class_=None, **attrs):
    """生成一个或多个HTML标签"""
    if class_ is not None:
        attrs['class'] = class_
    attr_pairs = (f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    attr_str = ''.join(attr_pairs)
    if content:
        return '\n'.join('<%s%s>%s</%s>' % (name, attr_str, c, name) for c in content)
    else:
        return '<%s%s />' % (name, attr_str)
    
print(tag('br'))
print(tag('p', 'hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'hello', 'world', class_='sidebar'))
print(tag(content='testing', name='img'))
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
print(tag(**my_tag))

<br />
<p>hello</p>
<p>hello</p>
<p>world</p>
<p id="33">hello</p>
<p class="sidebar">hello</p>
<p class="sidebar">world</p>
<img content="testing" />
<img cls="framed" src="sunset.jpg" title="Sunset Boulevard" />


定义函数时，如果想指定仅限关键字参数，要把它们放到前面有\*的参数后面。如果不想支持数量不定的定位参数，但是想支持仅限关键字参数，在签名中放一个\*。

In [17]:
def f(a, *, b):
    return a, b

print(f(1, b=2))
try:
    print(f(1, 2))
except Exception as e:
    print(e)

# 仅限位置参数
# / 之前的参数只能通过位置参数指定, / 之后的参数既可以通过位置参数指定，也可以通过关键字参数指定
def divmod(a, b, /):
    return (a // b, a % b)

(1, 2)
f() takes 1 positional argument but 2 were given


## 7.8 支持函数式编程的包

### 7.8.1 operator模块

operator模块为多个算术运算符提供了对应的函数，从而避免编写lambda a, b: a*b这种平凡的匿名函数。

In [ ]:
from functools import reduce
from operator import mul

def factorial(n):
    return reduce(mul, range(1, n+1))

operator模块中还有一类函数，能替代从序列中取出元素或读取对象属性的lambda表达式：itemgetter和attrgetter。

In [19]:
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]
from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

cc_name = itemgetter(1, 0)
for city in metro_data:
    print(cc_name(city))

('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833))
('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889))
('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
('Mexico City', 'MX', 20.142, (19.433333, -99.133333))
('New York-Newark', 'US', 20.104, (40.808611, -74.020386))
('JP', 'Tokyo')
('IN', 'Delhi NCR')
('MX', 'Mexico City')
('US', 'New York-Newark')
('BR', 'Sao Paulo')


attrgetter创建的函数会根据名称从对象中获取指定的属性。如果把多个属性名传给attrgetter，它也会返回提取的值构成的元组。
此外，如果参数名中包含.（点号），attrgetter会深入嵌套对象，获取指定的属性。

In [21]:
from collections import namedtuple
LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')
metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long)) for name, cc, pop, (lat, long) in metro_data]
print(metro_areas[0])

from operator import attrgetter
name_lat = attrgetter('name', 'coord.lat')

for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    print(name_lat(city))

Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))
('Sao Paulo', -23.547778)
('Mexico City', 19.433333)
('Delhi NCR', 28.613889)
('Tokyo', 35.689722)
('New York-Newark', 40.808611)


In [22]:
from operator import methodcaller
s = 'The time has come'
upcase = methodcaller('upper')
print(upcase(s))
hiphenate = methodcaller('replace', ' ', '-')
print(hiphenate(s))

THE TIME HAS COME
The-time-has-come


### 7.8.2 使用functools.partial冻结参数

functools模块提供了一系列高阶函数，其中functools.partial可以根据提供的可调用对象产生一个新的可调用对象，把原对象的某些参数固定。使用这个函数可以把接受一个或多个参数的函数改编成需要回调的API，这样参数更少。

In [23]:
from operator import mul
from functools import partial

triple = partial(mul, 3)
print(triple(7))

print(list(map(triple, range(1, 10))))

21
[3, 6, 9, 12, 15, 18, 21, 24, 27]


In [24]:
# 使用partial冻结一个位置参数和一个关键字参数
def tag(name, *content, cls=None, **attrs):
    """生成一个或多个HTML标签"""
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(f' {attr}="{value}"' for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join(f'<{name}{attr_str}>{c}</{name}>' for c in content)
    else:
        return f'<{name}{attr_str} />'

from functools import partial

picture = partial(tag, 'img', cls='pic-frame')

print(picture(src='wumpus.jpeg'))

<img class="pic-frame" src="wumpus.jpeg" />
